In [2]:
%run Jansen_And_Rit.py
# %run BOLD_Model.py
%run OptimiseFunctions.py

import numpy as np
import pickle
from skopt import gp_minimize
from skopt.space import Integer, Real

In [2]:
# Original A = 3.25 # excitatory, B = 22.0 # inhibitory, C = 135
# B > A. b < a. 
# b = 50, ad = 50, a = 100
# r_0, r_1, r_2 = 0.56
# mean_firing_threshold = 6
# max_firing_rate = 5

# A, B, C, a, ad, b, r_0, r_1, r_2, alpha, beta

# Alpha and beta 

# same sweep as https://doi.org/10.1371/journal.pcbi.1008737 - α 2 [0, 1], β 2 [0, 0.5] and r0 2 [0, 1].

# search_space = [
#     Integer(2, 15, name="A"),   
#     Integer(15, 45, name="B"),  
#     Integer(50, 170, name="C"), 
#     Integer(80, 160, name="a"),  
#     Integer(5, 80, name="ad"),   
#     Integer(5, 80, name="b"), 
#     Real(0.0, 1.0, name="r_0"), 
#     Real(0.0, 1.0, name="r_1"), 
#     Real(0.0, 1.0, name="r_2"), 
#     Real(0.0, 1.0, name="alpha"), 
#     Real(0.0, 0.5, name="beta") 
# ]


search_space = [
    Integer(1, 10, name="A"),   
    Integer(10, 40, name="B"),  
    Integer(90, 200, name="C"), 
    Integer(50, 150, name="a"),  
    Integer(10, 75, name="ad"),   
    Integer(10, 50, name="b"), 
    Real(0.0, 1.0, name="r_0"), 
    Real(0.0, 1.0, name="r_1"), 
    Real(0.0, 1.0, name="r_2"), 
    Real(0.0, 1.0, name="alpha"), 
    Real(0.0, 0.5, name="beta") 
]


########################################################################


In [14]:
from skopt import gp_minimize, forest_minimize, dummy_minimize, gbrt_minimize
from functools import partial
from skopt.plots import plot_convergence

n_calls = 120

# We will evaluate each model several times using a different seed for the random number generator. 
# Then compare the average performance of these models. This makes the comparison more robust against models that get “lucky”.

def run(minimizer, n_iter=2):
    return [minimizer(find_eeg_loss, search_space, n_calls=n_calls, random_state=n)
            for n in range(n_iter)]

# Random search
dummy_res = run(dummy_minimize)

# Gaussian processes
gp_res = run(gp_minimize)

# Random forest
rf_res = run(partial(forest_minimize, base_estimator="RF"))

# Extra trees
et_res = run(partial(forest_minimize, base_estimator="ET"))

# Gradient Boosted Trees
grbt_res = run(gbrt_minimize)

plot = plot_convergence(("dummy_minimize", dummy_res),
                        ("gp_minimize", gp_res),
                        ("forest_minimize('rf')", rf_res),
                        ("forest_minimize('et)", et_res),
                        ("gbrt_minimize", grbt_res),
                        title="Comparison of optimization techniques")

plot.legend(loc="best", prop={'size': 6}, numpoints=1)

6 35 184 135 51 25 0.29753460654447234 0.056712977317443194 0.27265629458011326 0.47766511732135 0.4060843643877467
Creating RawArray with float64 data, n_channels=62, n_times=2000
    Range : 0 ... 1999 =      0.000 ...     1.999 secs
Ready.
Effective window size : 2.000 (s)
5 22 182 84 52 25 0.9571551589530466 0.14035078041264518 0.8700872583584366 0.4736080452737106 0.4004553759898222
Creating RawArray with float64 data, n_channels=62, n_times=2000
    Range : 0 ... 1999 =      0.000 ...     1.999 secs
Ready.
Effective window size : 2.000 (s)
6 30 169 108 45 40 0.10590760718779216 0.47360041934665753 0.18633234332676 0.7369181771289582 0.10827517721218596


KeyboardInterrupt: 